### E-Commerce Customer Purchase Prediction
#### Phase 1: Data Split and Feature Engineering

In [33]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Step 1: Load Data

In [34]:
# Load the November 2019 dataset
data_url = "https://data.rees46.com/datasets/marketplace/2019-Nov.csv.gz"

print("Loading 2019-Nov.csv.gz (67.5M events)...")
df = pd.read_csv(data_url, compression='gzip')

print(f"Loaded {len(df):,} events")
print(f"Columns: {list(df.columns)}")
df.head()

Loading 2019-Nov.csv.gz (67.5M events)...
Loaded 67,501,979 events
Columns: ['event_time', 'event_type', 'product_id', 'category_id', 'category_code', 'brand', 'price', 'user_id', 'user_session']


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,NaN,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
3,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
4,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2


Step 2: Converting event_time to datetime and verifying we have the full November 2019 dataset

In [35]:
df['event_time'] = pd.to_datetime(df['event_time'])

print(f"Date range: {df['event_time'].min()} to {df['event_time'].max()}")
print(f"Total days: {(df['event_time'].max() - df['event_time'].min()).days + 1}")
print(f"Data types: {df.dtypes}")

Date range: 2019-11-01 00:00:00+00:00 to 2019-11-30 23:59:59+00:00
Total days: 30
Data types: event_time       datetime64[ns, UTC]
event_type                    object
product_id                     int64
category_id                    int64
category_code                 object
brand                         object
price                        float64
user_id                        int64
user_session                  object
dtype: object


Step 3: Define Sequential Time Windows for Proper time line Validation

**Training Set:**
- **Observation Window**: Nov 1-10, 2019 (10 days)
- **Prediction Window**: Nov 11-15, 2019 (5 days)

**Test Set:**
- **Observation Window**: Nov 16-25, 2019 (10 days)  
- **Prediction Window**: Nov 26-30, 2019 (5 days)

In [36]:
# Define TRAIN windows
TRAIN_OBS_START = pd.Timestamp('2019-11-01', tz='UTC')
TRAIN_OBS_END = pd.Timestamp('2019-11-10 23:59:59', tz='UTC')
TRAIN_PRED_START = pd.Timestamp('2019-11-11', tz='UTC')
TRAIN_PRED_END = pd.Timestamp('2019-11-15 23:59:59', tz='UTC')

# Define TEST windows
TEST_OBS_START = pd.Timestamp('2019-11-16', tz='UTC')
TEST_OBS_END = pd.Timestamp('2019-11-25 23:59:59', tz='UTC')
TEST_PRED_START = pd.Timestamp('2019-11-26', tz='UTC')
TEST_PRED_END = pd.Timestamp('2019-11-30 23:59:59', tz='UTC')

print(f"TRAIN Observation Window: {TRAIN_OBS_START.date()} to {TRAIN_OBS_END.date()}")
print(f"TRAIN Prediction Window:  {TRAIN_PRED_START.date()} to {TRAIN_PRED_END.date()}")
print(f"TEST Observation Window: {TEST_OBS_START.date()} to {TEST_OBS_END.date()}")
print(f"TEST Prediction Window:  {TEST_PRED_START.date()} to {TEST_PRED_END.date()}")

# Split TRAIN
df_train_obs = df[
    (df['event_time'] >= TRAIN_OBS_START) & 
    (df['event_time'] <= TRAIN_OBS_END)
].copy()

df_train_pred = df[
    (df['event_time'] >= TRAIN_PRED_START) & 
    (df['event_time'] <= TRAIN_PRED_END)
].copy()

# Split TEST
df_test_obs = df[
    (df['event_time'] >= TEST_OBS_START) & 
    (df['event_time'] <= TEST_OBS_END)
].copy()

df_test_pred = df[
    (df['event_time'] >= TEST_PRED_START) & 
    (df['event_time'] <= TEST_PRED_END)
].copy()

print(f"Train observation events: {len(df_train_obs):,}")
print(f"Train prediction events:  {len(df_train_pred):,}")
print(f"Test observation events:  {len(df_test_obs):,}")
print(f"Test prediction events:   {len(df_test_pred):,}")
print(f"Unique users in train observation: {df_train_obs['user_id'].nunique():,}")
print(f"Unique users in test observation:  {df_test_obs['user_id'].nunique():,}")

TRAIN Observation Window: 2019-11-01 to 2019-11-10
TRAIN Prediction Window:  2019-11-11 to 2019-11-15
TEST Observation Window: 2019-11-16 to 2019-11-25
TEST Prediction Window:  2019-11-26 to 2019-11-30
Train observation events: 17,285,581
Train prediction events:  15,306,266
Test observation events:  26,341,115
Test prediction events:   8,569,017
Unique users in train observation: 1,658,628
Unique users in test observation:  1,926,654


Step 4: Calculate Features from TRAIN Observation Window

All features come from the TRAIN observation window (Nov 1-10) to prevent data leakage

Feature categories:
1. Behavioral: events, views, cart actions
2. Product Exploration: unique products, categories, brands
3. Price Behavior: avg/max price, total viewed
4. Session Metrics: count, duration
5. RFM: recency, frequency, monetary value
6. Time patterns: hour/day behavior
7. Affinity: category/brand diversity

In [37]:
def feature_engineering(df_obs, df_pred, obs_end_timestamp, set_name=""):
    print(f"Engineering features for {set_name} set...")
    
    def count_by_type(event_series, event_type):
        return (event_series == event_type).sum()
    
    # BASE FEATURES: Behavioral + Product + Price + Session
    user_features = df_obs.groupby('user_id').agg(
        total_events=('event_type', 'count'),
        views=('event_type', lambda x: count_by_type(x, 'view')),
        cart_events=('event_type', lambda x: count_by_type(x, 'cart')),
        unique_products=('product_id', 'nunique'),
        unique_categories=('category_id', 'nunique'),
        unique_brands=('brand', 'nunique'),
        avg_price=('price', 'mean'),
        max_price=('price', 'max'),
        total_price_viewed=('price', 'sum'),
        sessions=('user_session', 'nunique'),
        first_event=('event_time', 'min'),
        last_event=('event_time', 'max')
    ).round(2)
    
    # DERIVED FEATURES
    user_features['events_per_session'] = (user_features['total_events'] / user_features['sessions']).round(2)
    user_features['cart_to_view_ratio'] = (user_features['cart_events'] / user_features['views'].replace(0, 1)).round(3)
    user_features['price_range'] = (user_features['max_price'] - user_features['avg_price']).round(2)
    user_features['session_duration_minutes'] = ((user_features['last_event'] - user_features['first_event']).dt.total_seconds() / 60).round(2)
    
    # RFM FEATURES
    user_features['recency_days'] = ((obs_end_timestamp - user_features['last_event']).dt.total_seconds() / (24 * 3600)).round(2)
    user_features['days_active'] = ((user_features['last_event'] - user_features['first_event']).dt.total_seconds() / (24 * 3600)).clip(lower=0.01).round(2)
    user_features['events_per_day'] = (user_features['total_events'] / user_features['days_active'].replace(0, 1)).round(2)
    user_features['sessions_per_day'] = (user_features['sessions'] / user_features['days_active'].replace(0, 1)).round(2)
    user_features['views_per_day'] = (user_features['views'] / user_features['days_active'].replace(0, 1)).round(2)
    # 50% price jump categorized as high value interaction
    user_features['high_value_interaction'] = (user_features['max_price'] > user_features['avg_price'] * 1.5).astype(int) 
    
    # SESSION VELOCITY FEATURES
    user_features['events_per_minute'] = (user_features['total_events'] / user_features['session_duration_minutes'].replace(0, 1)).round(3)
    user_features['views_per_session'] = (user_features['views'] / user_features['sessions'].replace(0, 1)).round(2)
    user_features['products_per_session'] = (user_features['unique_products'] / user_features['sessions'].replace(0, 1)).round(2)
    
    # TIME-BASED PATTERNS
    df_obs_copy = df_obs.copy()
    df_obs_copy['hour'] = df_obs_copy['event_time'].dt.hour
    df_obs_copy['dayofweek'] = df_obs_copy['event_time'].dt.dayofweek
    df_obs_copy['date'] = df_obs_copy['event_time'].dt.date
    
    time_features = df_obs_copy.groupby('user_id').agg(
        avg_hour_of_day=('hour', 'mean'),
        avg_day_of_week=('dayofweek', 'mean'),
        unique_days_active=('date', 'nunique'),
        unique_hours_active=('hour', 'nunique')
    ).round(2)
    
    user_features = user_features.join(time_features)
    user_features['primarily_weekend'] = (user_features['avg_day_of_week'] >= 5).astype(int)
    
    # AFFINITY FEATURES
    user_features['category_diversity'] = (user_features['unique_categories'] / user_features['total_events']).round(3)
    user_features['brand_diversity'] = (user_features['unique_brands'] / user_features['total_events']).round(3)
    user_features['brand_loyal'] = (user_features['unique_brands'] <= 2).astype(int)
    user_features['category_focused'] = (user_features['unique_categories'] <= 2).astype(int)
    
    # ADVANCED RATIOS
    user_features['products_per_category'] = (user_features['unique_products'] / user_features['unique_categories'].replace(0, 1)).round(2)
    user_features['brands_per_category'] = (user_features['unique_brands'] / user_features['unique_categories'].replace(0, 1)).round(2)
    user_features['price_stability'] = (user_features['avg_price'] / user_features['max_price'].replace(0, 1)).round(3)
    user_features['session_efficiency'] = (user_features['events_per_session'] / (user_features['session_duration_minutes'] / 60).replace(0, 1)).round(2)
    
    # TARGET VARIABLE
    users_who_purchased = set(df_pred[df_pred['event_type'] == 'purchase']['user_id'].unique())
    user_features['will_purchase'] = user_features.index.isin(users_who_purchased).astype(int)
    
    # Summary
    conversion_rate = user_features['will_purchase'].mean() * 100
    print(f"Users: {len(user_features):,}")
    # Excluding first_event, last_event, will_purchase
    print(f"Total columns: {len(user_features.columns)} (Features: {len(user_features.columns) - 3})")
    print(f"Conversion: {conversion_rate:.2f}%")
    print(f"Class balance: {user_features['will_purchase'].value_counts().to_dict()}")
    
    return user_features

Step 5: TRAIN set

In [38]:
user_features_train = feature_engineering(
    df_obs=df_train_obs,
    df_pred=df_train_pred,
    obs_end_timestamp=pd.Timestamp('2019-11-10 23:59:59', tz='UTC'),
    set_name="TRAIN"
)

train_overlap_check = df_train_obs['event_time'].max() < df_train_pred['event_time'].min()
print(f"TRAIN Data set Leakage Check:")
print(f"Observation ends: {df_train_obs['event_time'].max()}")
print(f"Prediction starts: {df_train_pred['event_time'].min()}")
assert train_overlap_check, "TRAIN: Observation and prediction windows overlap!"
user_features_train.head()

Engineering features for TRAIN set...
Users: 1,658,628
Total columns: 39 (Features: 36)
Conversion: 2.01%
Class balance: {0: 1625278, 1: 33350}
TRAIN Data set Leakage Check:
Observation ends: 2019-11-10 23:59:59+00:00
Prediction starts: 2019-11-11 00:00:00+00:00


,total_events,views,cart_events,unique_products,unique_categories,unique_brands,avg_price,max_price,total_price_viewed,sessions,...,primarily_weekend,category_diversity,brand_diversity,brand_loyal,category_focused,products_per_category,brands_per_category,price_stability,session_efficiency,will_purchase
user_id,,,,,,,,,,,,,,,,,,,,,
10300217,1,1,0,1,1,1,40.54,40.54,40.54,1,...,0,1.000,1.000,1,1,1.0,1.0,1.000,1.00,0
29515875,2,2,0,2,1,1,345.42,514.81,690.85,2,...,1,0.500,0.500,1,1,2.0,1.0,0.671,9.97,0
31198833,17,17,0,15,1,3,325.72,478.26,5537.17,1,...,0,0.059,0.176,0,1,15.0,3.0,0.681,31.02,0
49484535,5,5,0,2,2,2,46.33,55.34,231.63,5,...,1,0.400,0.400,1,1,1.0,1.0,0.837,1.49,0
50383810,1,1,0,1,1,1,199.14,199.14,199.14,1,...,1,1.000,1.000,1,1,1.0,1.0,1.000,1.00,0


Step 6: TEST set

In [39]:
user_features_test = feature_engineering(
    df_obs=df_test_obs,
    df_pred=df_test_pred,
    obs_end_timestamp=pd.Timestamp('2019-11-25 23:59:59', tz='UTC'),
    set_name="TEST"
)

test_overlap_check = df_test_obs['event_time'].max() < df_test_pred['event_time'].min()
print(f"TEST Data set Leakage Check:")
print(f"Observation ends: {df_test_obs['event_time'].max()}")
print(f"Prediction starts: {df_test_pred['event_time'].min()}")
assert test_overlap_check, "TEST: Observation and prediction windows overlap!"
user_features_test.head()

Engineering features for TEST set...
Users: 1,926,654
Total columns: 39 (Features: 36)
Conversion: 2.53%
Class balance: {0: 1877815, 1: 48839}
TEST Data set Leakage Check:
Observation ends: 2019-11-25 23:59:59+00:00
Prediction starts: 2019-11-26 00:00:00+00:00


,total_events,views,cart_events,unique_products,unique_categories,unique_brands,avg_price,max_price,total_price_viewed,sessions,...,primarily_weekend,category_diversity,brand_diversity,brand_loyal,category_focused,products_per_category,brands_per_category,price_stability,session_efficiency,will_purchase
user_id,,,,,,,,,,,,,,,,,,,,,
29515875,1,1,0,1,1,0,514.81,514.81,514.81,1,...,0,1.0,0.0,1,1,1.0,0.0,1.00,1.00,0
31198833,1,1,0,1,1,1,617.75,617.75,617.75,1,...,0,1.0,1.0,1,1,1.0,1.0,1.00,1.00,0
34916060,1,1,0,1,1,1,295.94,295.94,295.94,1,...,1,1.0,1.0,1,1,1.0,1.0,1.00,1.00,0
44378150,1,1,0,1,1,1,128.45,128.45,128.45,1,...,0,1.0,1.0,1,1,1.0,1.0,1.00,1.00,0
49484535,10,10,0,5,2,4,85.20,236.81,852.00,10,...,0,0.2,0.4,0,1,2.5,2.0,0.36,0.01,0


Step 7: Sampling 

In [40]:
import numpy as np

SAMPLE_FRACTION = 0.30

def stratified_sample(df, frac=0.30):
    np.random.seed(42)
    purchasers = df[df['will_purchase'] == 1].sample(frac=frac, random_state=42)
    non_purchasers = df[df['will_purchase'] == 0].sample(frac=frac, random_state=42)
    sampled = pd.concat([purchasers, non_purchasers])
    return sampled.sample(frac=1, random_state=42)

user_features_train = stratified_sample(user_features_train, SAMPLE_FRACTION)
user_features_test = stratified_sample(user_features_test, SAMPLE_FRACTION)

Step 8: Cohort Conversion Analysis
(Returning vs New Users)

In [41]:
train_users = set(df_train_obs['user_id'])
test_users = set(df_test_obs['user_id'])
overlap_users = train_users & test_users
new_users = test_users - train_users

print(f"Training period users: {len(train_users):,}")
print(f"Test period users: {len(test_users):,}")
print(f"Returning users: {len(overlap_users):,} ({len(overlap_users)/len(test_users)*100:.1f}%)")
print(f"New users: {len(new_users):,} ({len(new_users)/len(test_users)*100:.1f}%)")

user_features_test['user_cohort'] = 'New'
user_features_test.loc[user_features_test.index.isin(overlap_users), 'user_cohort'] = 'Returning'

cohort_stats = user_features_test.groupby('user_cohort')['will_purchase'].agg(['count', 'sum', 'mean'])
cohort_stats.columns = ['Total_Users', 'Purchasers', 'Avg_Conversion']
cohort_stats['Non_Purchasers'] = cohort_stats['Total_Users'] - cohort_stats['Purchasers']
cohort_stats['Conversion_Rate_%'] = cohort_stats['Avg_Conversion'] * 100
cohort_stats = cohort_stats[['Total_Users', 'Purchasers', 'Non_Purchasers', 'Conversion_Rate_%']].round(2)

print(cohort_stats)

returning_conv = cohort_stats.loc['Returning', 'Conversion_Rate_%']
new_conv = cohort_stats.loc['New', 'Conversion_Rate_%']
lift = returning_conv / new_conv if new_conv > 0 else 0

print(f"Returning users convert {lift:.1f}x better than new users")
print(f"({returning_conv:.2f}% vs {new_conv:.2f}%)")

Training period users: 1,658,628
Test period users: 1,926,654
Returning users: 649,983 (33.7%)
New users: 1,276,671 (66.3%)
             Total_Users  Purchasers  Non_Purchasers  Conversion_Rate_%
user_cohort                                                            
New               383147        6601          376546               1.72
Returning         194849        8051          186798               4.13
Returning users convert 2.4x better than new users
(4.13% vs 1.72%)


Step 9: Save the final data sets

In [42]:
# Prepare feature columns (exclude metadata and target)
feature_cols = [col for col in user_features_train.columns 
                if col not in ['will_purchase', 'first_event', 'last_event']]

# Create final splits
X_train = user_features_train[feature_cols]
y_train = user_features_train['will_purchase']
X_test = user_features_test[feature_cols]
y_test = user_features_test['will_purchase']

# Summary
print("TRAIN SET:")
print(f"Observation: {df_train_obs['event_time'].min().date()} to {df_train_obs['event_time'].max().date()}")
print(f"Prediction:  {df_train_pred['event_time'].min().date()} to {df_train_pred['event_time'].max().date()}")
print(f"Users: {len(X_train):,}")
print(f"Features: {X_train.shape[1]}")
print(f"Conversion: {y_train.mean()*100:.2f}%")

print("TEST SET:")
print(f"Observation: {df_test_obs['event_time'].min().date()} to {df_test_obs['event_time'].max().date()}")
print(f"Prediction:  {df_test_pred['event_time'].min().date()} to {df_test_pred['event_time'].max().date()}")
print(f"Users: {len(X_test):,}")
print(f"Features: {X_test.shape[1]}")
print(f"Conversion: {y_test.mean()*100:.2f}%")

# Save to disk
import os
os.makedirs('data/phase_1', exist_ok=True)

X_train.to_csv('data/phase_1/x_train.csv')
y_train.to_csv('data/phase_1/y_train.csv')
X_test.to_csv('data/phase_1/x_test.csv')
y_test.to_csv('data/phase_1/y_test.csv')

print("Datasets saved to data/phase_1/")
print(f"x_train.csv: {X_train.shape}")
print(f"y_train.csv: {y_train.shape}")
print(f"x_test.csv: {X_test.shape}")
print(f"y_test.csv: {y_test.shape}")

TRAIN SET:
Observation: 2019-11-01 to 2019-11-10
Prediction:  2019-11-11 to 2019-11-15
Users: 497,588
Features: 36
Conversion: 2.01%
TEST SET:
Observation: 2019-11-16 to 2019-11-25
Prediction:  2019-11-26 to 2019-11-30
Users: 577,996
Features: 36
Conversion: 2.53%
Datasets saved to data/phase_1/
x_train.csv: (497588, 36)
y_train.csv: (497588,)
x_test.csv: (577996, 36)
y_test.csv: (577996,)
